# Integrate well logs divided in different files

With this code you will be able to integrate DT and RHOB logs in one .las file.

In [1]:
import las
import numpy as np
import sys

In [2]:
las_files = ('perfis/1RJS_0051__RJ_1RJS_0051__RJ_BHC_00003.las', 'perfis/1RJS_0051__RJ_1RJS_0051__RJ_FDC_CNL_00001.las', 'perfis/1RJS_0051__RJ_1RJS_0051__RJ_FDC_CNL_GR_00004.las', 'perfis/1RJS_0051__RJ_1RJS_0051__RJ_IES_00005.las', 'perfis/1RJS_0051__RJ_1RJS_0051__RJ_ISF_BHC_GR_00002.las', 'perfis/1RJS_0051__RJ_1RJS_0051__RJ_SIGEO__013_00006.las')

In [3]:
N_files = len(las_files)
N_files

6

In [4]:
for i in range(N_files):
    globals()['log%s' % i] = las.LASReader(las_files[i])

In [5]:
log0.step

0.1524

In [6]:
np.set_printoptions(precision=4)

## Reading and assigning the logs:

In [7]:
def extract_logs(las_files):
    
    

    N_files = len(las_files) 

    DT = []
    RHOB = []
    DEPT = []
    DT_1 = []
    DT_11 = []
    DEPT_D = []
    DEPT_R = []
    RHOB_1 = []
    RHOB_11 = []
    data_DT = []
    data_RHOB = []

    for i in range(N_files):
        if 'Sonic' in open(las_files[i]).read():
            log = las.LASReader(las_files[i])

            DT_11 = np.append(DT_11, log.data['DT'])
            #DT_1 = np.append(DT_1, log.data['DT_1'])         # Add this command if your data have a collumns named DT_1
            DEPT_D = np.append(DEPT_D, log.data['DEPT'])
        
    null = np.where(DT_11 == -999.25)              # finding the indices of the null values
    #np.put(DT_11, [null], [DT_1[null]])         # not needed if you don't have DT_1 - it merges both DT logs
    null_2 = np.where(DT_11 == -999.25)         # finding null values in both logs
    DT_11[null_2] = 0.                        # replacing null values by 'nan'
    
    DT_2 = np.array([DT_11])
    DEPT_2 = np.array([DEPT_D])
    data_DT = np.concatenate((DEPT_2.T, DT_2.T), axis=1)
    data_DT = data_DT[np.argsort(data_DT[:, 0])]
    DT = data_DT[:,1]
    
    for i in range(N_files):
        if 'Bulk density' in open(las_files[i]).read():
            log = las.LASReader(las_files[i])
            
            RHOB_11 = np.append(RHOB_11, log.data['RHOB'])
            #RHOB_1 = np.append(RHOB_1, log.data['RHOB_1'])      # Add this command if your data have a collumns named RHOB_1
            DEPT_R = np.append(DEPT_R, log.data['DEPT'])
            
    null = np.where(RHOB_11 == -999.25)              # finding the indices of the null values
    #np.put(RHOB_11, [null], [RHOB_1[null]])         # not needed if you don't have DT_1 - it merges both RHOB logs
    null_2 = np.where(RHOB_11 == -999.25)         # finding null values in both logs
    RHOB_11[null_2] = 0.                        # replacing null values by 'nan'

    RHOB_2 = np.array([RHOB_11])
    DEPT_2 = np.array([DEPT_R])
    data_RHOB = np.concatenate((DEPT_2.T, RHOB_2.T), axis=1)
    data_RHOB = data_RHOB[np.argsort(data_RHOB[:, 0])]
    RHOB = data_RHOB[:,1]
    
    if DEPT_R.size > DEPT_D.size:
        DEPT = data_RHOB[:,0]
        D =  np.empty(DEPT.size,)
        D.fill(np.nan)
        idx_DT = np.asarray(np.where(DT >= 0))
        D[(idx_DT[:])] = DT
        DT = D

    elif DEPT_D.size > DEPT_R.size:
        DEPT = data_DT[:,0]
        R =  np.empty(DEPT.size,)
        R.fill(np.nan)
        idx_RHOB = np.asarray(np.where(RHOB >= 0))
        R[(idx_RHOB[:])] = RHOB
        RHOB = R
        
    null_3 = np.where(DT == 0.)
    DT[null_3] = np.nan 
    data_DT[null_3] = np.nan 
        
    null_3 = np.where(RHOB == 0.)
    RHOB[null_3] = np.nan  
    data_RHOB[null_3] = np.nan 
    
    return DEPT, DT, RHOB, data_DT, data_RHOB

In [8]:
DEPT, DT, RHOB, data_DT, data_RHOB = extract_logs(las_files)

In [9]:
np.amin(data_DT)

nan

In [10]:
data_RHOB

array([[  5.7429e+02,   1.8759e+00],
       [  5.7444e+02,   1.8448e+00],
       [  5.7460e+02,   1.8203e+00],
       ..., 
       [  3.5366e+03,   2.6824e+00],
       [  3.5368e+03,   2.6799e+00],
       [  3.5369e+03,   2.6827e+00]])

In [11]:
DEPT.size

19488

## Reading the start and end of the depth:

In [12]:
DEPT_start = DEPT[0]
DEPT_end = DEPT[-1]

In [13]:
print DEPT_start

574.292


In [14]:
print DEPT_end

3532.8332


## Exporting integrated logs to a txt file

In [15]:
np.savetxt('DT_int.txt', data_DT, fmt='%s', delimiter='    ', newline='\n', header='', footer='', comments='# ') 

In [16]:
np.savetxt('RHOB_int.txt', data_RHOB, fmt='%s', delimiter='    ', newline='\n', header='', footer='', comments='# ') 